# Visualize Database
- Following thread "http://www.austintaylor.io/d3/python/pandas/2016/02/01/create-d3-chart-python-force-directed/"
- Attempting to visualize the dvdrentals sample postgres database

## The Network Structure
- A dictionary with two lists, nodes and links.
- links contains the relationships between nodes
- nodes contains each individual node

```json
{
  "nodes":  [
    { "name": "desktop", "group":  1},
    { "name": "desktop/apples.txt", "group":  1},
    { "name": "desktop/pineapple/apples.txt", "group":  1},
    { "name": "desktop/bananas.txt", "group":  1}
  ],

  "links":  [
    { "source":  1,  "target":  0,  "value":  5555 },
    { "source":  2,  "target":  0,  "value":  1 },
    { "source":  3,  "target":  0,  "value": 1 }
  ]
}
```

## Setup

### Modules

In [1]:
import os
import pandas
import json
import psycopg2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Postgres SQL code generating the csv below

```SQL

-- Drop table if it exists

DROP TABLE
	test_data

-- Create new table containing column and table information from dvdrentals database

SELECT 
	*
INTO 
	test_data 
FROM (
SELECT
	table_schema,
	table_name,
	column_name
FROM
	information_schema.columns
WHERE
	table_schema = 'public' AND
	table_name IN ('actor','address','category','city','country',
				   'customer','film','film_actor','film_category','inventory',
				   'language','payment','rental','staff','store')
) AS subset

-- Export to disk
COPY
	(SELECT * FROM test_data)
TO
	'/users/danielcorcoran/desktop/github_repos/python_nb_networks/raw data/dvdrentals_column_data.csv'
DELIMITER
	','
CSV HEADER

```

### Data File Path 

In [2]:
data_path = "/users/danielcorcoran/desktop/github_repos/python_nb_networks/raw data/dvdrentals_column_data.csv"

### Read in data, add columns

In [3]:
raw_data = pandas.read_csv(data_path)

In [4]:
raw_data.head()

,table_schema,table_name,column_name
0,public,staff,staff_id
1,public,staff,first_name
2,public,staff,last_name
3,public,staff,address_id
4,public,staff,email


## Manipulate raw data into desired format

### Union

In [5]:
subset1 = raw_data[["table_schema", "table_name"]]
subset2 = raw_data[["table_name", "column_name"]]

In [6]:
subset1.head()

,table_schema,table_name
0,public,staff
1,public,staff
2,public,staff
3,public,staff
4,public,staff


In [7]:
subset2.head()

,table_name,column_name
0,staff,staff_id
1,staff,first_name
2,staff,last_name
3,staff,address_id
4,staff,email


In [8]:
subset1.columns = ["source", "target"]
subset2.columns = ["source", "target"]

In [9]:
subset1["structure"] = "table"
subset2["structure"] = "column"

# subset1["colour_code"] = "#4ABBDB"
# subset2["colour_code"] = "#FFAA00"

subset1["node_size"] = 6
subset2["node_size"] = 4

In [10]:
data = pandas.concat([subset1, subset2], axis = 0)

In [11]:
data.head()

,source,target,structure,node_size
0,public,staff,table,6
1,public,staff,table,6
2,public,staff,table,6
3,public,staff,table,6
4,public,staff,table,6


In [12]:
data.shape

(172, 4)

In [13]:
data.drop_duplicates(inplace = True)

In [14]:
data.reset_index(drop = True, inplace = True)

### Probe database for column sizes

In [15]:
connection = psycopg2.connect("host=localhost dbname=dvdrentals user=danielcorcoran")

In [16]:
for index in range(data.shape[0]):
    
    structure = data.loc[index, "structure"]
    
    if structure == "column":
        
        column_name = data.loc[index, "target"]
        table_name = data.loc[index, "source"]

        print("column name is {}, table name is {}".format(column_name, table_name))
        cursor = connection.cursor()
        cursor.execute("SELECT SUM(pg_column_size({})) AS SIZE FROM {}".format(column_name, table_name))
        records = cursor.fetchall()
        record_dataframe = pandas.DataFrame(records)
        record_dataframe
        
        value = record_dataframe.iloc[0,0]
        
        data.loc[index, "size"] = value
        
        cursor.close()

column name is staff_id, table name is staff
column name is first_name, table name is staff
column name is last_name, table name is staff
column name is address_id, table name is staff
column name is email, table name is staff
column name is store_id, table name is staff
column name is active, table name is staff
column name is username, table name is staff
column name is password, table name is staff
column name is last_update, table name is staff
column name is picture, table name is staff
column name is category_id, table name is category
column name is name, table name is category
column name is last_update, table name is category
column name is film_id, table name is film_category
column name is category_id, table name is film_category
column name is last_update, table name is film_category
column name is country_id, table name is country
column name is country, table name is country
column name is last_update, table name is country
column name is actor_id, table name is actor
col

In [17]:
data.head()

,source,target,structure,node_size,size
0,public,staff,table,6,NaN
1,public,category,table,6,NaN
2,public,film_category,table,6,NaN
3,public,country,table,6,NaN
4,public,actor,table,6,NaN


In [18]:
for index in range(data.shape[0]):
    
    structure = data.loc[index, "structure"]
    
    if structure == "table":
        
        table_name = data.loc[index, "target"]
        
        aggregate_size = data[data["source"] == table_name]["size"].sum()
        
        data.loc[index, "size"] = aggregate_size

#### Setting the bins column

In [19]:
data["bins"] = pandas.cut(data["size"], bins = 7)
data["bins"] = data["bins"].astype(str)

#### Show unique bins

In [20]:
print(data["bins"].unique())

['(-574.118, 82304.571]' '(329212.286, 411514.857]'
 '(493817.429, 576120.0]' '(82304.571, 164607.143]'
 '(164607.143, 246909.714]']


#### Create sizing for bins

In [21]:
colours = ['#fdd49e',
'#fdbb84',
'#fc8d59',
'#ef6548',
'#d7301f',
'#990000']

In [22]:
offset = 3

for index in range(data.shape[0]):
    bin_text = data.loc[index, "bins"]
    
    if bin_text == '(-574.118, 82304.571]':
        data.loc[index, "bins_node_size"] = offset + 3
        data.loc[index, "html_colour"] = colours[0]
        
    elif bin_text == '(82304.571, 164607.143]':
        data.loc[index, "bins_node_size"] = offset + 4
        data.loc[index, "html_colour"] = colours[1]
        
    elif bin_text == '(164607.143, 246909.714]':
        data.loc[index, "bins_node_size"] = offset + 5
        data.loc[index, "html_colour"] = colours[2]
        
    elif bin_text == '(329212.286, 411514.857]':
        data.loc[index, "bins_node_size"] = offset + 6
        data.loc[index, "html_colour"] = colours[3]
        
    elif bin_text == '(493817.429, 576120.0]':
        data.loc[index, "bins_node_size"] = offset + 7
        data.loc[index, "html_colour"] = colours[4]

## Build Json

### Helper Functions

In [23]:
def get_unique_nodes_from_list(list_of_dictionaries, seeking_key = "name"):
   
    the_list = list(map(lambda dictionary: dictionary[seeking_key], list_of_dictionaries))
    
    return the_list    

### Set source and target column headers

In [24]:
source_header = "source"
target_header = "target"

### Convert items to strings

In [25]:
data[source_header] = data[source_header].astype(str)

In [26]:
data[target_header] = data[target_header].astype(str)

## Process

In [27]:
nodes_list = []
links_list = []

row_count = data.shape[0]

### Iterate through each row in the dataset

In [28]:
for index in range(row_count):
    
    #print index
    print("Processing index {}".format(index))
    
    # At the row level, store the target text, source text and other optional parameters such as colour, node_size etc
    target_text = data.loc[index, target_header]
    source_text = data.loc[index, source_header]
    html_colour = data.loc[index, "html_colour"]
    size = str(data.loc[index, "bins_node_size"])
    max_node_size = data["bins_node_size"].max()
    home_node_size = max_node_size + 3
    
    print("Target text is {}, source text is {}".format(target_text, source_text))
    
    # Create a unique list of current nodes in nodes list
    current_nodes = get_unique_nodes_from_list(nodes_list)
    
    # Check if target and source text exists in the current nodes list. if they dont create them
    if target_text not in current_nodes:
        nodes_list.append({"name":target_text, "colour": html_colour, "size": size})
        
    if source_text not in current_nodes:
        nodes_list.append({"name":source_text, "colour": colours[4], "size": home_node_size})
        
    # Recreate unique nodes list after the changes made above
    updated_nodes = get_unique_nodes_from_list(nodes_list)

    # Locate the index of the 
    target_node_index = updated_nodes.index(target_text)
    source_node_index = updated_nodes.index(source_text)
    
    links_list.append({"source":source_node_index, 
                        "target":target_node_index,
                        "value":10})

Processing index 0
Target text is staff, source text is public
Processing index 1
Target text is category, source text is public
Processing index 2
Target text is film_category, source text is public
Processing index 3
Target text is country, source text is public
Processing index 4
Target text is actor, source text is public
Processing index 5
Target text is language, source text is public
Processing index 6
Target text is inventory, source text is public
Processing index 7
Target text is payment, source text is public
Processing index 8
Target text is rental, source text is public
Processing index 9
Target text is city, source text is public
Processing index 10
Target text is store, source text is public
Processing index 11
Target text is film, source text is public
Processing index 12
Target text is address, source text is public
Processing index 13
Target text is film_actor, source text is public
Processing index 14
Target text is customer, source text is public
Processing index 15

### Create dictionary storing links_list and nodes_list together

In [29]:
json_data = {"links": links_list,"nodes":nodes_list}

## Export Json

### Convert python dictionary to json string

In [30]:
json_dump = json.dumps(json_data, indent=1, sort_keys=True)

### Export to filename 'pcap_export.json' to be used in index.html

In [31]:
json_out = open("json/dvdrental_database_network_2.json", "w")
json_out.write(json_dump)
json_out.close()